In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/Shareddrives/CS229Buddies/project')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import collections
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold, LeaveOneOut
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve, f1_score, plot_precision_recall_curve
from scipy.stats import pearsonr
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import operator
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
df = pd.read_csv('data/GSE113957_fpkm.txt', sep = '\t')
df = df.T
transcript_ids = list(df.loc['Transcript ID'])

df = df.drop(['Transcript ID', 'chr', 'start', 'end', 'strand', 'Length', 'Copies', 'Annotation/Divergence'], axis=0)
df['pids'] = [column.lower().split('_')[0] for column in df.index]
hgps = set(['139', '141', '197', '198', '199', '200', '201', '202', '203', '204'])
control_df = df[~df.pids.isin(hgps)]
hgps_df = df[df.pids.isin(hgps)]
control_df = control_df.drop(['pids'], axis = 1)
hgps_df = hgps_df.drop(['pids'], axis = 1)

control_df.columns = transcript_ids
hgps_df.columns = transcript_ids

total_df = control_df.append(hgps_df)

In [ ]:
annot=pd.read_csv("data/uniprot.tab", sep='\t')

In [ ]:
name_d=pd.Series(annot.Entry.values,index=annot["yourlist:M202105245C475328CEF75220C360D524E9D456CE20BD46G"]).to_dict()

In [ ]:
total_df=total_df[[x for x in total_df.columns if x  in name_d.keys()]]

In [ ]:
total_df.columns=[name_d[x] for x in total_df.columns if x  in name_d.keys()]
total_df

,Q2QL34,Q9UHB7,P0C6A0,Q8NE62,Q32Q52,Q8TF47,O15054,Q6PJP8,A6NJW4,A5JUU3,A0A024R3G4,Q9H410,Q6ZWJ8,P63208,Q9P2N4,O43395,Q8N546,B4DXW2,P14653,D6R9V8,A0A0C4DFX9,B4DN02,P56373,O95071,Q13057,Q96PC3,Q96CW1,A6NKU9,Q5W0V3,A6NKF7,P61225,Q9BYG8,Q9NVQ4,O95255,Q58A44,P29373,Q5EG05,Q6IMI6,Q96GA3,Q7Z599,...,P0C7Q6,O15130,Q9UBP5,Q9UK76,Q9BWS9,Q6PJ22,Q5DID0,A0A0S2Z4Z5,P25686,E5KMT6,Q96FJ0,Q2PZI1,Q7L2H7,B4DNP6,Q6YFQ2,Q12947,A2RRD8,P63241,P47871,Q9BYD9,A0A140VJN5,A2RQD9,Q9H5P4,V9HWF8,Q9BQA9,B4DP66,Q9HD67,Q8IWK6,Q13239,Q9H944,Q8NHY3,Q49AI2,B6ZDM2,Q9H3H9,Q7Z6Z6,Q7L099,Q8N441,Q9NY28,A0A024RDC7,Q6ZT89
101_19yr_Female_Caucasian,0.133,16.98,0.387,0.115,0,5.5,1.308,2.143,0,0,5.127,14.031,0.018,68.975,0.62,19.216,2.564,7.417,0,52.032,4.525,3.126,0,12.938,14.259,0.541,277.734,0,5.743,0,3.595,0,9.665,0,0.797,148.414,0.733,0,14.825,0,...,0,0.517,0.016,63.004,16.377,11.96,0.017,14.666,16.771,7.705,4.727,10.064,44.714,0,0.077,3.535,2.652,225.407,0,1.994,0,0.03,0.29,1.875,11.888,27.75,17.19,2.77,0,9.433,0.014,3.978,17.081,0,0,4.349,29.242,0.04,6.139,0
102_19yr_Male_Caucasian,0.213,16.772,0.168,0.077,0,6.473,1.357,1.952,0,0,3.701,13.255,0,61.827,0.211,11.628,1.296,7.865,0,36.853,3.693,3.514,0,11.535,11.706,0.202,284.64,0,5.608,0,2.291,0.07,6.522,0,0.773,315.897,0.189,0,9.692,0,...,0,0.289,0,51.358,23.395,15.45,0,10.447,13.185,4.05,5.36,9.991,29.864,0,0.029,2.435,2.613,205.338,0,2.873,0.018,0.022,0.441,2.624,12.583,21.449,18.545,4.02,0,6.454,0,3.378,14.215,0,0,5.642,31.561,0,5.603,0
103_19yr_Male_Caucasian,0.054,16.206,0.41,0.284,0,6.77,1.495,1.881,0.046,0,3.356,9.516,0.009,67.533,0.326,12.417,1.84,7.482,0,42.325,4.726,2.08,0,10.953,14.44,0.182,301.299,0,6.187,0,2.293,0,8.685,0,1.418,266.717,1.021,0,11.141,0,...,0,1.134,0,56.853,21.832,16.853,0,13.672,15.413,5.758,4.526,10.105,34.743,0,0.195,3.389,2.404,219.336,0,2.488,0.022,0.038,0.712,2.508,14.085,23.936,15.557,3.193,0,8.573,0,5.841,15.057,0.037,0,6.132,43.446,0,6.03,0
104_19yr_Male_Caucasian,0.066,15.345,0.148,0.304,0,5.773,1.688,1.927,0.056,0.039,2.455,9.884,0,62.057,0.503,12.878,1.867,8.941,0,31.844,5.389,3.217,0,11.372,13.423,0.183,315.602,0,5.836,0,2.931,0.018,8.092,0,0.612,125.909,0.974,0,9.815,0,...,0,1.488,0,50.358,22.499,13.521,0.01,13.59,16.273,4.885,5.067,8.32,31.115,0,0.059,3.466,1.996,183.929,0,3.15,0.018,0.034,0.573,2.339,14.824,29.59,15.571,3.297,0,8.485,0,2.237,14.247,0,0.022,3.752,39.371,0,5.048,0
105_20yr_Male_Caucasian,0.156,16.873,0.252,0.094,0,5.052,1.384,2.815,0,0,5.745,13.131,0.011,63.981,0.242,14.719,1.448,7.345,0,43.607,4.981,2.663,0,11.345,14.707,0.398,283.42,0.016,6.723,0,1.785,0.018,6.667,0,0.934,66.549,0.264,0,13.647,0.018,...,0,1.152,0,70.406,17.125,12.348,0,11.55,12.744,6.255,4.777,9.909,39.632,0,0,1.822,2.437,239.498,0,1.517,0.037,0.081,0.097,0.794,11.617,20.223,15.222,4.057,0,8.247,0,3.537,14.088,0.091,0,4.833,18.592,0,3.794,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200_8yr6mos_M_Unknown,0.147,17.866,0.362,0.056,0,5.792,1.862,2.007,0,0.012,4.203,7.575,0.013,77.791,0.513,13.059,2.361,7.151,0,47.306,5.091,2.069,0,11.013,15.425,0.812,281.264,0,7.597,0,2.003,0.055,7.107,0.042,0.597,208.561,0.356,0,16.667,0.011,...,0,0.653,0.022,57.206,20.837,12.528,0,13.327,16.896,6.065,9.195,6.842,38.84,0,0.036,1.363,2.133,232.839,0,2.039,0.033,0.111,0.379,2.413,15.212,26.09,18.202,3.685,0,9.156,0.03,5.069,12.207,0.109,0,4.022,28.202,0.014,4.88,0
201_6ys11mos_F_Unknown,0.18,18.088,0.435,0.054,0,6.769,1.994,2.484,0,0,4.206,7.528,0.018,79.464,0.255,16.109,2.63,7.716,0,56.502,4.912,2.017,0,10.773,16.151,0.882,281.178,0,7.754,0,2.52,0,9.059,0.04,0.359,91.103,0.724,0,14.871,0.031,...,0,0.769,0,73.589,21.468,13.013,0,15.398,19.125,6.401,5.299,5.952,41.196,0,0.138,1.247,2.52,225.573,0,1.361,0.011,0.02,0.462,1.355,11.575,2

In [ ]:
# !pip install bioservices

In [ ]:
from bioservices import UniProt, QuickGO, KEGGParser

In [ ]:
go = QuickGO()

In [ ]:
go.Annotation()

{'numberOfHits': 969032682,
 'pageInfo': {'current': 1, 'resultsPerPage': 100, 'total': 9690327},
 'results': [{'assignedBy': 'InterPro',
   'date': '20210515',
   'evidenceCode': 'ECO:0000256',
   'extensions': None,
   'geneProductId': 'UniProtKB:A0A000',
   'goAspect': 'molecular_function',
   'goEvidence': 'IEA',
   'goId': 'GO:0030170',
   'goName': None,
   'id': 'UniProtKB:A0A000!153722185',
   'name': None,
   'qualifier': 'enables',
   'reference': 'GO_REF:0000002',
   'symbol': 'moeA5',
   'synonyms': None,
   'targetSets': None,
   'taxonId': 67581,
   'taxonName': None,
   'withFrom': [{'connectedXrefs': [{'db': 'InterPro', 'id': 'IPR004839'}]},
    {'connectedXrefs': [{'db': 'InterPro', 'id': 'IPR010961'}]}]},
  {'assignedBy': 'InterPro',
   'date': '20210515',
   'evidenceCode': 'ECO:0000256',
   'extensions': None,
   'geneProductId': 'UniProtKB:A0A000',
   'goAspect': 'biological_process',
   'goEvidence': 'IEA',
   'goId': 'GO:0033014',
   'goName': None,
   'id': 'Uni

In [ ]:
def get_annotations(Id, aspect="P", taxonId=9606):
  annots=set()
  pairs=set()
  for x in go.Annotation(geneProductId="Q2QL34", taxonId=9606, aspect="P")['results']:
    annots.add(x["goId"])
  return annots

In [ ]:
gene_annot=dict()
i=0
for x in total_df.columns:
  if i%500==0:
    print(i)
 
  i+=1
  gene_annot[x]=get_annotations(x)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500


In [ ]:
gene_annot['P04350']

({('GO:0010730', None), ('GO:0072593', None), ('GO:1901029', None)},
 {'GO:0010730', 'GO:0072593', 'GO:1901029'})

In [ ]:
import collections
annot_genes=collections.defaultdict(set)
true_genes_annot = collections.defaultdict(set)
for k,v in gene_annot.items():
  for go in v[1]:
    annot_genes[go].add(k)
    true_genes_annot[k].add(go)

In [ ]:
import pickle
with open('data/genes_annot.pickle', 'wb') as handle:
    pickle.dump(true_genes_annot, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
with open('data/annot_genes.pickle', 'wb') as handle:
    pickle.dump(annot_genes, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
with open('data/annot_genes.pickle', 'rb') as handle:
    annot_genes = pickle.load(handle)



In [ ]:
df = pd.read_csv('data/GSE113957_fpkm.txt', sep = '\t')
df = df.T
transcript_ids = list(df.loc['Transcript ID'])

df = df.drop(['Transcript ID', 'chr', 'start', 'end', 'strand', 'Length', 'Copies', 'Annotation/Divergence'], axis=0)
df['pids'] = [column.lower().split('_')[0] for column in df.index]
hgps = set(['139', '141', '197', '198', '199', '200', '201', '202', '203', '204'])
control_df = df[~df.pids.isin(hgps)]
hgps_df = df[df.pids.isin(hgps)]
control_df = control_df.drop(['pids'], axis = 1)
hgps_df = hgps_df.drop(['pids'], axis = 1)

control_df.columns = transcript_ids
hgps_df.columns = transcript_ids

total_df = control_df.append(hgps_df)

In [ ]:
total_df

,NM_173803,NM_014423,NM_001103167,NR_134623,NR_024490,NM_018397,NM_001037671,NR_106859,NM_133458,NM_001080424,NM_001271816,NR_027283,NR_073417,NR_033323,NM_001195545,NM_014879,NM_006918,NM_001145318,NR_109942,NR_135665,NM_001135914,NM_170679,NR_027000,NM_001318781,NM_004698,NM_017554,NM_015603,NR_136640,NM_002144,NR_102367,NM_001282763,NM_005663,NM_153213,NM_002559,NR_038194,NM_015902,NR_024111,NM_001042529,NM_001039569,NM_001025205,...,NR_038285,NM_207333,NM_001143760,NR_133006,NR_051989,NM_003525,NM_000160,NM_032487,NR_110936,NR_038990,NM_001079910,NR_110223,NM_138319,NM_024895,NR_134558,NM_004683,NM_001100408,NR_040535,NR_049865,NM_001318099,NM_012334,NM_145290,NM_006748,NR_033295,NR_030191,NM_001305457,NM_139285,NM_004332,NM_022495,NR_027141,NR_029666,NM_080390,NM_001177675,NR_125786,NM_001291993,NM_021923,NM_017417,NM_004354,NR_024278,NM_145282
101_19yr_Female_Caucasian,0.133,16.98,0.387,0,0.626,0.115,0,31.531,5.5,1.308,2.143,0.026,0,1.159,0,0,5.127,14.031,0,0.219,0.018,68.975,0.015,0.62,19.216,2.564,7.417,0,0,0.359,52.032,4.525,3.126,0,0,12.938,23.253,14.259,0.541,277.734,...,0.851,2.652,225.407,0,0,0,0,1.994,6.163,0.03,0,0,0.03,0.29,0.244,1.875,11.888,0.071,0,27.75,17.19,2.77,0,0,0,9.433,0.014,3.978,17.081,6.395,0,0,0,0,4.349,29.242,0.04,6.139,0.528,0
102_19yr_Male_Caucasian,0.213,16.772,0.168,0,0.701,0.077,0,27.992,6.473,1.357,1.952,0.029,0,1.487,0,0,3.701,13.255,0,0.28,0,61.827,0,0.211,11.628,1.296,7.865,0,0,0.494,36.853,3.693,3.514,0,0,11.535,28.562,11.706,0.202,284.64,...,1.203,2.613,205.338,0,0,0.219,0,2.873,4.459,0,0.018,0,0.022,0.441,0.091,2.624,12.583,0.147,0,21.449,18.545,4.02,0,0,0,6.454,0,3.378,14.215,7.04,0,0,0,0,5.642,31.561,0,5.603,1.1,0
103_19yr_Male_Caucasian,0.054,16.206,0.41,0,1.135,0.284,0,25.915,6.77,1.495,1.881,0,0,2.187,0.046,0,3.356,9.516,0,0.403,0.009,67.533,0,0.326,12.417,1.84,7.482,0,0,0.631,42.325,4.726,2.08,0,0,10.953,28.39,14.44,0.182,301.299,...,1.281,2.404,219.336,0.07,0,0,0,2.488,4.339,0,0.022,0,0.038,0.712,0.058,2.508,14.085,0.125,0,23.936,15.557,3.193,0,0,0,8.573,0,5.841,15.057,6.55,0,0.037,0,0,6.132,43.446,0,6.03,0.972,0
104_19yr_Male_Caucasian,0.066,15.345,0.148,0,0.685,0.304,0,18.776,5.773,1.688,1.927,0,0,1.89,0.056,0.039,2.455,9.884,0,0.306,0,62.057,0,0.503,12.878,1.867,8.941,0,0,0.476,31.844,5.389,3.217,0,0,11.372,25.176,13.423,0.183,315.602,...,1.237,1.996,183.929,0.084,0,0,0,3.15,4.482,0,0.018,0,0.034,0.573,0.374,2.339,14.824,0.26,0,29.59,15.571,3.297,0,0,0,8.485,0,2.237,14.247,5.598,0,0,0.022,0,3.752,39.371,0,5.048,1.461,0
105_20yr_Male_Caucasian,0.156,16.873,0.252,0.09,0.806,0.094,0,18.481,5.052,1.384,2.815,0,0,1.529,0,0,5.745,13.131,0,0.165,0.011,63.981,0,0.242,14.719,1.448,7.345,0,0,0.453,43.607,4.981,2.663,0,0,11.345,29.6,14.707,0.398,283.42,...,0.945,2.437,239.498,0.043,0,0,0,1.517,5.535,0,0.037,0,0.081,0.097,0.024,0.794,11.617,0.042,0,20.223,15.222,4.057,0,0,0,8.247,0,3.537,14.088,5.844,0,0.091,0,0,4.833,18.592,0,3.794,1.238,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200_8yr6mos_M_Unknown,0.147,17.866,0.362,0,0.591,0.056,0,32.428,5.792,1.862,2.007,0,0,1.39,0,0.012,4.203,7.575,0,0.176,0.013,77.791,0,0.513,13.059,2.361,7.151,0,0,0.949,47.306,5.091,2.069,0,0,11.013,22.692,15.425,0.812,281.264,...,1.571,2.133,232.839,0,0,0.069,0,2.039,4.341,0,0.033,0,0.111,0.379,0.071,2.413,15.212,0.075,0,26.09,18.202,3.685,0,0,0,9.156,0.03,5.069,12.207,5.699,0,0.109,0,0,4.022,28.202,0.014,4.88,1.333,0
201_6ys11mos_F_Unknown,0.18,18.088,0.435,0,0.665,0.054,0,37.091,6.769,1.994,2.484,0,0,1.892,0,0,4.206,7.528,0,0.222,0.018,79.464,0.01,0.255,16.109,2.63,7.716,0,0,1.901,56.502,4.912,2.017,0,0,10.773,23.606,16.151,0.882,281.178,...,1.059,2.52,225.573,0.025,0,0.33,0,1.361,7.153,0,0.011,0,0.02,0.462,0.027,1.355,11.575

In [ ]:
with open('transcript_ids.txt', 'w') as f:
    for transcript in transcript_ids:
        f.write("%s\n" % transcript)

In [ ]:
import mygene
mg = mygene.MyGeneInfo()
symbol = mg.querymany('NR_027283', scopes='refseq')[0]['symbol']
go.Annotation(geneProductId=symbol, taxonId=9606)

querying 1-1...done.


WARNING [bioservices:quickGO:112]:  status is not ok with 
WARNING [bioservices:quickGO:112]:  status is not ok with 


Finished.


400

In [ ]:
mg.querymany('NR_134623', scopes='refseq')[0]

querying 1-1...done.
Finished.


{'_id': '105373021',
 '_score': 36.543243,
 'entrezgene': '105373021',
 'name': 'uncharacterized LOC105373021',
 'query': 'NR_134623',
 'symbol': 'LOC105373021',
 'taxid': 9606}